# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [202]:
!pip install scikit-multilearn
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import metrics
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score


In [203]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [204]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
# loading the dataset

# train_df = pd.read_csv('drive/MyDrive/train.csv')
# test_df = pd.read_csv('drive/MyDrive/test.csv')
# test_labels = pd.read_csv('drive/MyDrive/test_labels.csv')


train_df = pd.read_csv('q4_data/train.csv')
test_df = pd.read_csv('q4_data/test.csv')
test_labels = pd.read_csv('q4_data/test_labels.csv')

#dataset understanding
# print(train_df.sample(5))
# print(test_df.sample(5))
print(test_labels.sample(5))




                      id  toxic  severe_toxic  obscene  threat  insult  \
84620   8d24865a8a56bcfb     -1            -1       -1      -1      -1   
82605   89c8f937678c75eb     -1            -1       -1      -1      -1   
116964  c329de4bc562d881      1             0        0       0       1   
59419   62c6b7a18d0b79a2      0             0        0       0       0   
101229  a8eea8e9d69907cb      0             0        0       0       0   

        identity_hate  
84620              -1  
82605              -1  
116964              0  
59419               0  
101229              0  


In [206]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
# print(train_df[cols_target].sum())


Total rows in test is 153164
Total rows in train is 159571


In [207]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [208]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [209]:
print(train_df)
# print()
print(test_df)



                      id                                       comment_text  \
0       0000997932d777bf  explanation why the edits made under my userna...   
1       000103f0d9cfb60f  d aww he matches this background colour i am s...   
2       000113f07ec002fd  hey man i am really not trying to edit war it ...   
3       0001b41b1c6bb37e  more i cannot make any real suggestions on imp...   
4       0001d958c54c6e35  you sir are my hero any chance you remember wh...   
...                  ...                                                ...   
159566  ffe987279560d7ff  and for the second time of asking when your vi...   
159567  ffea4adeee384e90  you should be ashamed of yourself that is a ho...   
159568  ffee36eab5c267c9  spitzer umm theres no actual article for prost...   
159569  fff125370e4aaaf3  and it looks like it was actually you who put ...   
159570  fff46fc426af1f9a  and i really do not think you understand i cam...   

        toxic  severe_toxic  obscene  threat  insul

In [210]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, )
print(test_X.shape, )






(159571,)
(153164,)


In [211]:
Y1 =train_df.toxic
Y2 =train_df.severe_toxic
Y3 =train_df.obscene
Y4 =train_df.threat
Y5 =train_df.insult
Y6 =train_df.identity_hate

In [212]:
Y = train_df[["toxic", 	"severe_toxic" ,	"obscene" 	,"threat" ,	"insult" 	,"identity_hate"]]
Y_test = test_labels[["toxic", 	"severe_toxic" ,	"obscene" 	,"threat" ,	"insult" 	,"identity_hate"]]


In [213]:
Y

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [214]:
Y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...
153159,-1,-1,-1,-1,-1,-1
153160,-1,-1,-1,-1,-1,-1
153161,-1,-1,-1,-1,-1,-1
153162,-1,-1,-1,-1,-1,-1


In [215]:
Y_test.shape

(153164, 6)

In [216]:
that_index = Y_test[Y_test['toxic'] ==-1].index


In [217]:
that_index2 = that_index

In [218]:
print(that_index)
print(len(that_index))


Int64Index([     0,      1,      2,      3,      4,      6,      8,      9,
                10,     12,
            ...
            153148, 153152, 153153, 153157, 153158, 153159, 153160, 153161,
            153162, 153163],
           dtype='int64', length=89186)
89186


In [219]:
# Y_test.drop(that_index, inplace = True)
Y_test = Y_test.drop(that_index)

In [220]:
Y_test.shape

(63978, 6)

In [221]:
Y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0,0,0,0,0,0
7,0,0,0,0,0,0
11,0,0,0,0,0,0
13,0,0,0,0,0,0
14,0,0,0,0,0,0
...,...,...,...,...,...,...
153150,0,0,0,0,0,0
153151,0,0,0,0,0,0
153154,0,0,0,0,0,0
153155,1,0,1,0,1,0


In [222]:
Y1_test =Y_test.toxic
Y2_test =Y_test.severe_toxic
Y3_test =Y_test.obscene
Y4_test =Y_test.threat
Y5_test =Y_test.insult
Y6_test =Y_test.identity_hate

In [223]:
# print(Y1_test)

In [224]:
print(that_index)
print(len(that_index))


Int64Index([     0,      1,      2,      3,      4,      6,      8,      9,
                10,     12,
            ...
            153148, 153152, 153153, 153157, 153158, 153159, 153160, 153161,
            153162, 153163],
           dtype='int64', length=89186)
89186


In [225]:
test_X.shape

(153164,)

In [226]:
test_X

0         yo bitch ja rule is more succesful then you wi...
1                   from rfc the title is fine as it is imo
2                            sources zawe ashton on lapland
3         if you have a look back at the source the info...
4                 i do not anonymously edit articles at all
                                ...                        
153159    i totally agree this stuff is nothing but too ...
153160    throw from out field to home plate does it get...
153161    okinotorishima categories i see your changes a...
153162    one of the founding nations of the eu germany ...
153163    stop already your bullshit is not welcome here...
Name: comment_text, Length: 153164, dtype: object

In [227]:
test_X = test_X.drop(that_index)

In [228]:
test_X.shape

(63978,)

In [229]:
X

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i am s...
2         hey man i am really not trying to edit war it ...
3         more i cannot make any real suggestions on imp...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself that is a ho...
159568    spitzer umm theres no actual article for prost...
159569    and it looks like it was actually you who put ...
159570    and i really do not think you understand i cam...
Name: comment_text, Length: 159571, dtype: object

In [230]:
# Y1_test =train_df.toxic
# Y2 =train_df.severe_toxic
# Y3 =train_df.obscene
# Y4 =train_df.threat
# Y5 =train_df.insult
# Y6 =train_df.identity_hate

In [231]:
# from google.colab import drive
# drive.mount('/content/drive')

Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [232]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [233]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [234]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


In [235]:
# # Remove These Lines
# X = X[:10000]
# test_X=test_X[:10000]
# # 

In [236]:
# # Remove These Lines
# Y= Y[:10000]
# Y_test = Y_test[:10000]


In [237]:
# Y1 =Y1[:10000]
# Y2 =Y2[:10000]
# Y3 =Y3[:10000]
# Y4 =Y4[:10000]
# Y5 =Y5[:10000]
# Y6 =Y6[:10000]

# Y1_test =Y1_test[:10000]
# Y2_test =Y2_test[:10000]
# Y3_test =Y3_test[:10000]
# Y4_test =Y4_test[:10000]
# Y5_test =Y5_test[:10000]
# Y6_test =Y6_test[:10000]

In [238]:
#Obtain X_test and X_train 
# X=X.cuda()
X_train=[]
for each in X:
  this_embed= get_use_embedding(each)
  X_train.append(this_embed)

X_test=[]
for each in test_X:
  this_embed= get_use_embedding(each)
  X_test.append(this_embed)
  

In [239]:
# X=pd.DataFrame(X_train)

In [240]:
print(X.shape)
print(Y1.shape)
print(Y2.shape)
print(Y3.shape)
print(Y4.shape)
print(Y5.shape)
print(Y6.shape)
print(type(Y1))



(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
<class 'pandas.core.series.Series'>


In [241]:


# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)



In [242]:
# x_train1, x_test1, y_train1, y_test1 = train_test_split(X, Y1, test_size = 0.2)
# x_train2, x_test2, y_train2, y_test2 = train_test_split(X, Y2, test_size = 0.2)
# x_train3, x_test3, y_train3, y_test3 = train_test_split(X, Y3, test_size = 0.2)
# x_train4, x_test4, y_train4, y_test4 = train_test_split(X, Y4, test_size = 0.2)
# x_train5, x_test5, y_train5, y_test5 = train_test_split(X, Y5, test_size = 0.2)
# x_train6, x_test6, y_train6, y_test6 = train_test_split(X, Y6, test_size = 0.2)

In [243]:
X_train=pd.DataFrame(X_train)
Y_train=pd.DataFrame(Y)


In [244]:
Y1_train=pd.DataFrame(Y1)
Y2_train=pd.DataFrame(Y2)
Y3_train=pd.DataFrame(Y3)
Y4_train=pd.DataFrame(Y4)
Y5_train=pd.DataFrame(Y5)
Y6_train=pd.DataFrame(Y6)


In [245]:

X_test=pd.DataFrame(X_test)


In [246]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(10000, 512)
(10000, 6)
(10000, 512)
(10000, 6)


In [247]:
# print(X_train.shape)
# print(Y_train.shape)
# print(type(X_test))

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [248]:
#calculate the accuracies obtained, u can use sklearn metrics


In [249]:
# Naive Bayes classifier 
# nb1 = GaussianNB()
# nb2 = GaussianNB()
# nb3 = GaussianNB()
# nb4 = GaussianNB()
# nb5 = GaussianNB()
# nb6 = GaussianNB()

# nb1.fit(x_train1, y_train1)
# nb2.fit(x_train2, y_train2)
# nb3.fit(x_train3, y_train3)
# nb4.fit(x_train4, y_train4)
# nb5.fit(x_train5, y_train5)
# nb6.fit(x_train6, y_train6)



classifier = BinaryRelevance(GaussianNB())
classifier.fit(X_train, Y_train)
predictions = classifier.predict(X_test)
accuracy = accuracy_score(Y_test,predictions)

print("Accuracy of Naive Bayes is : ",accuracy)


Accuracy of Naive Bayes is :  0.8063


In [250]:
# ac1=nb1.score(x_test1, y_test1)
# ac2=nb2.score(x_test2, y_test2)
# ac3=nb3.score(x_test3, y_test3)
# ac4=nb4.score(x_test4, y_test4)
# ac5=nb5.score(x_test5, y_test5)
# ac6=nb6.score(x_test6, y_test6)
# print("Naive Bayes score1: ",ac1)
# print("Naive Bayes score2: ",ac2)
# print("Naive Bayes score3: ",ac3)
# print("Naive Bayes score4: ",ac4)
# print("Naive Bayes score5: ",ac5)
# print("Naive Bayes score6: ",ac5)

# print("Naive Bayes Final Accuracy: ",(ac1+ac2+ac3+ac4+ac5+ac6)/6)



In [251]:
#Support Vector Machines
# clf1 = svm.SVC(kernel='linear')
# clf2 = svm.SVC(kernel='linear')
# clf3 = svm.SVC(kernel='linear')
# clf4 = svm.SVC(kernel='linear')
# clf5 = svm.SVC(kernel='linear')
# clf6 = svm.SVC(kernel='linear')

# clf1.fit(x_train1, y_train1)
# clf2.fit(x_train2, y_train2)
# clf3.fit(x_train3, y_train3)
# clf4.fit(x_train4, y_train4)
# clf5.fit(x_train5, y_train5)
# clf6.fit(x_train6, y_train6)

# y_pred1 = clf1.predict(x_test1)
# y_pred2 = clf2.predict(x_test2)
# y_pred3 = clf3.predict(x_test3)
# y_pred4 = clf4.predict(x_test4)
# y_pred5 = clf5.predict(x_test5)
# y_pred6 = clf6.predict(x_test6)

# ac1= metrics.accuracy_score(y_test1, y_pred1)
# ac2= metrics.accuracy_score(y_test2, y_pred2)
# ac3= metrics.accuracy_score(y_test3, y_pred3)
# ac4= metrics.accuracy_score(y_test4, y_pred4)
# ac5= metrics.accuracy_score(y_test5, y_pred5)
# ac6= metrics.accuracy_score(y_test6, y_pred6)


# print("Accuracy1:",ac1)
# print("Accuracy2:",ac2)
# print("Accuracy3:",ac3)
# print("Accuracy4:",ac4)
# print("Accuracy5:",ac5)
# print("Accuracy6:",ac6)
# print("Final SVM Accuracy :" ,(ac1+ac2+ac3+ac4+ac5+ac6)/6 )


classifier2 = BinaryRelevance(svm.SVC(kernel='linear'))
classifier2.fit(X_train, Y_train)
predictions = classifier2.predict(X_test)
accuracy = accuracy_score(Y_test,predictions)

print("Accuracy of SVM is : ",accuracy)





Accuracy of SVM is :  0.8952


Which classifier of the three performed better, and why do you think so ?

Ans: MLP performed better because it is a Neural network and is more powerful than the other two.
It learns the weight by back propogation and can predict the result with more accuracy.

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [252]:
def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output

def sigmoid_derivative(input):
    return sigmoid(input) * (1.0 - sigmoid(input))

def train_network(features,label,weights,bias,learning_rate,epochs):
  for epoch in range(epochs):
    dot_prod = np.dot(features, weights) + bias
    # using sigmoid
    preds = sigmoid(dot_prod)
    # # Error
    # print(preds.shape)
    # print(label.shape)
    errors = preds - label
    deriva_cost_funct = errors
    # print(deriva_cost_funct.shape)
    deriva_preds = sigmoid_derivative(preds)
    # print(deriva_preds.shape)
    deriva_product = deriva_cost_funct * deriva_preds
    #update the weights
    feature=features.T
    weights = weights -  np.dot(feature, deriva_product) * learning_rate
    loss = errors.sum()
    # print(loss)
    for i in deriva_product:
      bias = bias -  i * learning_rate

  return weights,bias

In [253]:
np.random.seed(10)
features  = X_train
label = Y1_train.values.reshape(Y1_train.shape[0],1)
print(label.shape)
weights = np.random.rand(512,1)
# weights2 = np.random.rand(10,1)
# bias1 = np.random.randint(10)
# bias2 = np.random.randint(1)
bias = np.random.rand(1)
learning_rate = 0.0004
epochs = 100

(10000, 1)


In [254]:
# train_network(x_train1,y_train1,weights1,bias,learning_rate,epochs)
weights_final, bias_final = train_network(features,label,weights,bias,learning_rate,epochs)

In [255]:
#  MLP 


In [256]:
print(weights_final.shape)

(512, 1)


In [257]:
# print(y_test1.shape)


# print(type(y_test1))

# y1=y_test1.to_numpy()
# print(type(y1))


In [258]:
# y2=pd.DataFrame(y1)


In [259]:
# y2

# y3=y2.to_numpy()
# print(y3.shape)

# for i in range(len(y_test1)) :
#   print(y_test1[i])

In [260]:
# inputs = [[86,104]]
# inputs=pd.DataFrame(inputs).to_numpy()
# print(inputs.shape)
# inputs=inputs.T
# dot_prod = np.dot(inputs, weights_final) + bias_final
# preds = sigmoid(dot_prod) >= 1/2
# preds


# x_train1=pd.DataFrame(x_train1)
# o=[]
# x1=x_train1.to_numpy()
# for i in range(len(x_train1)) :
#   # print(x_test1.loc[i])
  
#   x2=pd.DataFrame(x1[i])
#   xo=x2.to_numpy()
#   x3=xo.T
#   # print(x3.shape)
#   # print(x1[0].shape)
#   # print(p.shape)
#   dot_prod = np.dot(x3, weights_final) + bias_final
#   preds = sigmoid(dot_prod) >= 1/2
#   if(preds[0][0]==False):
#     o.append(0)
#   else:
#     o.append(1)


# print(len(o))






In [261]:
# print(type(y_train1))

In [262]:
# print(o)

# q=pd.DataFrame(o)

# print(type(q))

# # b=q.to_numpy()
# # print(type(b))

# # print(b.shape)

# print(type(y_train1))

# z=pd.DataFrame(y_train1)

# print(type(z))


# a=z.to_numpy()
# print(a.shape)

In [263]:
# z

In [264]:
# z.rename(columns = {'toxic':'0'}, inplace = True)



In [265]:
# z

In [266]:
# q

In [267]:
# (z.reset_index(drop=True) == q.reset_index(drop=True))

# u=q.reset_index(drop=True)

# print(u)

# v=z.reset_index(drop=True)

# print(v)
# acc=np.sum(q.reset_index(drop=True).equals(
#     z.reset_index(drop=True))) /q.size



# print(acc)

In [268]:
# acc=np.sum(u==v)/u.size
# print(acc)

# fg=u.values.tolist()
# print(fg)

# print(type(u))

# gh=v.values.tolist()
# print(gh)

# print(type(u))




In [269]:
# print(fg[0]==gh[0])
# print(fg[1]==gh[1])
# count=0

# for i in range(len(fg)):
#   if(fg[i]==gh[i]):
#     count+=1

In [270]:
# print(count/len(fg))

In [271]:
# print(type(x_train1))
# x_train1=x_train1.to_numpy()


In [272]:
o=[]
x1=X_test.to_numpy()
for i in range(len(X_test)) :
  # print(x_test1.loc[i])
  
  x2=pd.DataFrame(x1[i])
  xo=x2.to_numpy()
  x3=xo.T
  # print(x3.shape)
  # print(x1[0].shape)
  # print(p.shape)
  dot_prod = np.dot(x3, weights_final) + bias_final
  preds = sigmoid(dot_prod) >= 1/2
  if(preds[0][0]==False):
    o.append(0)
  else:
    o.append(1)


print(len(o))


10000


In [273]:
print(o)

q=pd.DataFrame(o)

print(type(q))

# b=q.to_numpy()
# print(type(b))

# print(b.shape)

print(type(Y1_test))

z=pd.DataFrame(Y1_test)

print(type(z))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [274]:
z.rename(columns = {'toxic':'0'}, inplace = True)


In [275]:
# z

In [276]:
# q

In [277]:
u=q.reset_index(drop=True)

print(u)

v=z.reset_index(drop=True)

print(v)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]


In [278]:
fg=u.values.tolist()
print(fg)

print(type(u))

gh=v.values.tolist()
print(gh)

print(type(u))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [279]:
count=0

for i in range(len(fg)):
  if(fg[i]==gh[i]):
    count+=1

In [280]:
print(count)

9060


In [281]:
a1=(count/len(fg))

0.906


In [282]:
# print(type(x_train1))
# print()
# print(x_train1.shape)
# # x_train1=x_train1.T
# print(x_train1[0].shape)
# # x_train1[0]=x_train1[0].T
# print(x_train1[0])

In [283]:
# print(type(x_train1[0]))
# x_train1=pd.DataFrame(x_train1[0])
# # print(x_train1[0].shape)



In [284]:
# print(type(x_train1))
# x_train1

In [285]:
# xo=x_train1.to_numpy()

In [286]:
# print(xo.shape)

In [287]:
# x1=xo.T
# print(x1.shape)

In [288]:
# print(type(x1))

In [289]:
# xp=pd.DataFrame(x1)


In [290]:
# xp

In [291]:
np.random.seed(10)
features  = X_train
label = Y2_train.values.reshape(Y2_train.shape[0],1)
print(label.shape)
weights = np.random.rand(512,1)
# weights2 = np.random.rand(10,1)
# bias1 = np.random.randint(10)
# bias2 = np.random.randint(1)
bias = np.random.rand(1)
learning_rate = 0.0004
epochs = 100

(10000, 1)


In [292]:
# train_network(x_train1,y_train1,weights1,bias,learning_rate,epochs)
weights_final, bias_final = train_network(features,label,weights,bias,learning_rate,epochs)

In [293]:
print(weights_final.shape)

(512, 1)


In [294]:
o=[]
x1=X_test.to_numpy()
for i in range(len(X_test)) :
  # print(x_test1.loc[i])
  
  x2=pd.DataFrame(x1[i])
  xo=x2.to_numpy()
  x3=xo.T
  # print(x3.shape)
  # print(x1[0].shape)
  # print(p.shape)
  dot_prod = np.dot(x3, weights_final) + bias_final
  preds = sigmoid(dot_prod) >= 1/2
  if(preds[0][0]==False):
    o.append(0)
  else:
    o.append(1)


print(len(o))


10000


In [295]:
print(o)

q=pd.DataFrame(o)

print(type(q))

# b=q.to_numpy()
# print(type(b))

# print(b.shape)

print(type(Y2_test))

z=pd.DataFrame(Y2_test)

print(type(z))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [296]:
z.rename(columns = {'severe_toxic':'0'}, inplace = True)


In [297]:
u=q.reset_index(drop=True)

print(u)

v=z.reset_index(drop=True)

print(v)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]


In [298]:
fg=u.values.tolist()
print(fg)

print(type(u))

gh=v.values.tolist()
print(gh)

print(type(u))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [299]:
count=0

for i in range(len(fg)):
  if(fg[i]==gh[i]):
    count+=1

In [300]:
print(count)

9944


In [301]:
a2=(count/len(fg))

0.9944


In [302]:
np.random.seed(10)
features  = X_train
label = Y3_train.values.reshape(Y3_train.shape[0],1)
print(label.shape)
weights = np.random.rand(512,1)
# weights2 = np.random.rand(10,1)
# bias1 = np.random.randint(10)
# bias2 = np.random.randint(1)
bias = np.random.rand(1)
learning_rate = 0.0004
epochs = 100

(10000, 1)


In [303]:
# train_network(x_train1,y_train1,weights1,bias,learning_rate,epochs)
weights_final, bias_final = train_network(features,label,weights,bias,learning_rate,epochs)

In [304]:
print(weights_final.shape)

(512, 1)


In [305]:
o=[]
x1=X_test.to_numpy()
for i in range(len(X_test)) :
  # print(x_test1.loc[i])
  
  x2=pd.DataFrame(x1[i])
  xo=x2.to_numpy()
  x3=xo.T
  # print(x3.shape)
  # print(x1[0].shape)
  # print(p.shape)
  dot_prod = np.dot(x3, weights_final) + bias_final
  preds = sigmoid(dot_prod) >= 1/2
  if(preds[0][0]==False):
    o.append(0)
  else:
    o.append(1)


print(len(o))


10000


In [306]:
print(o)

q=pd.DataFrame(o)

print(type(q))

# b=q.to_numpy()
# print(type(b))

# print(b.shape)

print(type(Y3_test))

z=pd.DataFrame(Y3_test)

print(type(z))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [307]:
z.rename(columns = {'obscene':'0'}, inplace = True)


In [308]:
u=q.reset_index(drop=True)

print(u)

v=z.reset_index(drop=True)

print(v)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]


In [309]:
fg=u.values.tolist()
print(fg)

print(type(u))

gh=v.values.tolist()
print(gh)

print(type(u))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [310]:
count=0

for i in range(len(fg)):
  if(fg[i]==gh[i]):
    count+=1

In [311]:
print(count)

9435


In [312]:
a3=(count/len(fg))

0.9435


In [313]:
np.random.seed(10)
features  = X_train
label = Y4_train.values.reshape(Y4_train.shape[0],1)
print(label.shape)
weights = np.random.rand(512,1)
# weights2 = np.random.rand(10,1)
# bias1 = np.random.randint(10)
# bias2 = np.random.randint(1)
bias = np.random.rand(1)
learning_rate = 0.0004
epochs = 100

(10000, 1)


In [314]:
# train_network(x_train1,y_train1,weights1,bias,learning_rate,epochs)
weights_final, bias_final = train_network(features,label,weights,bias,learning_rate,epochs)

In [315]:
print(weights_final.shape)

(512, 1)


In [316]:
o=[]
x1=X_test.to_numpy()
for i in range(len(X_test)) :
  # print(x_test1.loc[i])
  
  x2=pd.DataFrame(x1[i])
  xo=x2.to_numpy()
  x3=xo.T
  # print(x3.shape)
  # print(x1[0].shape)
  # print(p.shape)
  dot_prod = np.dot(x3, weights_final) + bias_final
  preds = sigmoid(dot_prod) >= 1/2
  if(preds[0][0]==False):
    o.append(0)
  else:
    o.append(1)


print(len(o))


10000


In [317]:
print(o)

q=pd.DataFrame(o)

print(type(q))

# b=q.to_numpy()
# print(type(b))

# print(b.shape)

print(type(Y4_test))

z=pd.DataFrame(Y4_test)

print(type(z))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [318]:
z.rename(columns = {'threat':'0'}, inplace = True)


In [319]:
u=q.reset_index(drop=True)

print(u)

v=z.reset_index(drop=True)

print(v)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]


In [320]:
fg=u.values.tolist()
print(fg)

print(type(u))

gh=v.values.tolist()
print(gh)

print(type(u))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [321]:
count=0

for i in range(len(fg)):
  if(fg[i]==gh[i]):
    count+=1

In [322]:
print(count)

9963


In [323]:
a4=(count/len(fg))

0.9963


In [324]:
np.random.seed(10)
features  = X_train
label = Y5_train.values.reshape(Y5_train.shape[0],1)
print(label.shape)
weights = np.random.rand(512,1)
# weights2 = np.random.rand(10,1)
# bias1 = np.random.randint(10)
# bias2 = np.random.randint(1)
bias = np.random.rand(1)
learning_rate = 0.0004
epochs = 100

(10000, 1)


In [325]:
# train_network(x_train1,y_train1,weights1,bias,learning_rate,epochs)
weights_final, bias_final = train_network(features,label,weights,bias,learning_rate,epochs)

In [326]:
print(weights_final.shape)

(512, 1)


In [327]:
o=[]
x1=X_test.to_numpy()
for i in range(len(X_test)) :
  # print(x_test1.loc[i])
  
  x2=pd.DataFrame(x1[i])
  xo=x2.to_numpy()
  x3=xo.T
  # print(x3.shape)
  # print(x1[0].shape)
  # print(p.shape)
  dot_prod = np.dot(x3, weights_final) + bias_final
  preds = sigmoid(dot_prod) >= 1/2
  if(preds[0][0]==False):
    o.append(0)
  else:
    o.append(1)


print(len(o))


10000


In [328]:
print(o)

q=pd.DataFrame(o)

print(type(q))

# b=q.to_numpy()
# print(type(b))

# print(b.shape)

print(type(Y5_test))

z=pd.DataFrame(Y5_test)

print(type(z))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [329]:
z.rename(columns = {'insult':'0'}, inplace = True)


In [330]:
u=q.reset_index(drop=True)

print(u)

v=z.reset_index(drop=True)

print(v)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]


In [331]:
fg=u.values.tolist()
print(fg)

print(type(u))

gh=v.values.tolist()
print(gh)

print(type(u))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [332]:
count=0

for i in range(len(fg)):
  if(fg[i]==gh[i]):
    count+=1

In [333]:
print(count)

9489


In [334]:
a5=(count/len(fg))

0.9489


In [335]:
np.random.seed(10)
features  = X_train
label = Y6_train.values.reshape(Y6_train.shape[0],1)
print(label.shape)
weights = np.random.rand(512,1)
# weights2 = np.random.rand(10,1)
# bias1 = np.random.randint(10)
# bias2 = np.random.randint(1)
bias = np.random.rand(1)
learning_rate = 0.0004
epochs = 100

(10000, 1)


In [336]:
# train_network(x_train1,y_train1,weights1,bias,learning_rate,epochs)
weights_final, bias_final = train_network(features,label,weights,bias,learning_rate,epochs)

In [337]:
print(weights_final.shape)

(512, 1)


In [338]:
o=[]
x1=X_test.to_numpy()
for i in range(len(X_test)) :
  # print(x_test1.loc[i])
  
  x2=pd.DataFrame(x1[i])
  xo=x2.to_numpy()
  x3=xo.T
  # print(x3.shape)
  # print(x1[0].shape)
  # print(p.shape)
  dot_prod = np.dot(x3, weights_final) + bias_final
  preds = sigmoid(dot_prod) >= 1/2
  if(preds[0][0]==False):
    o.append(0)
  else:
    o.append(1)


print(len(o))


10000


In [339]:
print(o)

q=pd.DataFrame(o)

print(type(q))

# b=q.to_numpy()
# print(type(b))

# print(b.shape)

print(type(Y6_test))

z=pd.DataFrame(Y6_test)

print(type(z))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [340]:
z.rename(columns = {'identity_hate':'0'}, inplace = True)


In [341]:
u=q.reset_index(drop=True)

print(u)

v=z.reset_index(drop=True)

print(v)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]
      0
0     0
1     0
2     0
3     0
4     0
...  ..
9995  0
9996  0
9997  0
9998  0
9999  0

[10000 rows x 1 columns]


In [342]:
fg=u.values.tolist()
print(fg)

print(type(u))

gh=v.values.tolist()
print(gh)

print(type(u))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [343]:
count=0

for i in range(len(fg)):
  if(fg[i]==gh[i]):
    count+=1

In [344]:
print(count)

9878


In [345]:
a6=(count/len(fg))

0.9878


In [346]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed

def initialise_weights():
  return
  

def loss_function():
  return 


def forward_layer():
  return 


def backward_propogation():
  return 




In [347]:
def train():
  pass

In [348]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def predict():
  pass



In [349]:
#report accuracies on the test dataset 

print("Accuracy of MLP is :", (a1+a2+a3+a4+a5+a6)/6.0)



Now, compare your implemented MLP with the sklearn MLP layer.

In [350]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier
# hidden_layer_sizes=(150,100,50)

classifier3 = BinaryRelevance(MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1))
classifier3.fit(X_train, Y_train)
predictions = classifier3.predict(X_test)
accuracy = accuracy_score(Y_test,predictions)

print("Accuracy of MLP is : ",accuracy)


# classifier = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)



Accuracy of MLP is :  0.8772


Which MLP performed better? 
Understand and Analyse.

Sklearn's MLP performed better, because in scratch MLP i have taken only one hidden layer for simplicity of the code.
In the Sklearn's MLP layer i have taken three hidden layers of sizes 150 ,100 and 50 Neurons respectively that's why it performed better. 